#BERTopic on UDC Dataset
In this notebook I have implemented Topic Modeling on Urdu News Data using a transformer based topic modeling technique BERTopic.

## Mounting Google Drive
If the dataset is on Google Drive then you have to mount over google drive with collaboratory.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive




#Installing required dependencies
**One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.**

In [ ]:
!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.1/154.1 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 18.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.9 MB/s eta 0:00:00
  Created wheel for hdbscan: filename=hdbscan-0.8.33-cp310-cp310-linux_x86_64.whl size=3039286 sha256=7a70762f48a9fb23e250cac3e13756b16ef38a53c4c7ca4e1492caa528b49004
  Stored in dire

In [ ]:
!pip install -U sentence-transformers

In [ ]:
!pip install --upgrade tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.5 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling te


# Importing required dependencies
We will import numpy, pandas and re, bertopic, gensim library for now. other libraries will be imported in the notebook later.

Pandas will be used to create a Dataframe and handle the csv file. Numpy will be used for the faster computation of arrays to save time. re library will be used for the cleaning of data. gensim library is used to get coherence score. bertopic is used to train bertopic on UDC dataset with using pretrained language models

In [ ]:
import pandas as pd
import numpy as np
import re
from bertopic import BERTopic
from gensim.models.coherencemodel import CoherenceModel
import gensim.corpora as corpora
#optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

##DataFrame
We also performed experiment on publically available dataset for topic modelling that consist of 5 categories=>Business, Entertainment, Health,weird and Sports. After cleaning this news, total 1008 rows saved into another new column Clean_Data for experiments.The link to the dataset is given below:

[Dataset](https://github.com/Mubashar331/Urdu-corpus)




In [ ]:
#load UNTM Dataset
df = pd.read_csv('/content/drive/MyDrive/Clean_BigCorpus.csv', encoding='utf-8')

In [ ]:
print(len(df))
df.head()

1008


,Document,Category,Clean_Data
0,﻿اسلام آباد: ایسوسی ایشن آف چارٹرڈ سرٹیفائی...,Business,اسلام آباد ایسوسی ایشن آف چارٹرڈ سرٹیفائیڈا...
1,اسلام آباد: ایف بی آرنے 1 ارب30کروڑ روپے سے ...,Business,اسلام آباد ایف بی آرنے اربکروڑ روپے سے زائد...
2,کراچی: بزنس مین گروپ کے چیئرمین اور کراچی چیم...,Business,کراچی بزنس مین گروپ کے چیئرمین اور کراچی چیمب...
3,کراچی: ٹرانسپورٹ اور پٹرولیم مصنوعات کی درآم...,Business,کراچی ٹرانسپورٹ اور پٹرولیم مصنوعات کی درآمد...
4,بجٹ کی آمد آمد ہے۔ ایکسپریس میڈیا گروپ کی ی...,Business,بجٹ کی آمد آمد ہے۔ ایکسپریس میڈیا گروپ کی ی...


## Preprocessing of Data
Data was cleaned that saved into column "Clean_Data"  We further remove some unnecessary words.
Stopwords are common words that are often filtered out during text processing in natural language processing (NLP) tasks. These words are considered to have little or no value in conveying the actual meaning of the text. We take list of 401 stopwords for topic modelling.

In [ ]:
df['Clean_Data'] = df['Clean_Data'].str.replace('شیئرٹویٹشیئرای میلتبصرے مزید شیئر', '')

In [ ]:
df['Clean_Data'] = df['Clean_Data'].str.replace('کیلیے', '')

In [ ]:
def remove_nonbreaking_space(text):
    return re.sub(r'\xa0', ' ', text)

df['Clean_Data'] = df['Clean_Data'].apply(remove_nonbreaking_space)

In [ ]:
documents = df['Clean_Data'].values.tolist()
print((documents[1]))
doc1=documents[1]

 اسلام آباد ایف بی آرنے  اربکروڑ روپے سے زائد مالیت کے گڈز ڈکلیئریشن فنڈ کے استعمال  خصوصی سیکریٹریٹ قائم کرنے اور ایف بی آر ہیڈ کوارٹرز اورکسٹمز ہاؤس کراچی میں قائم ڈیٹا سینٹرزکی اپ گریڈیشن کرنے کا فیصلہ کیا ہے۔  ایکسپریس کو دستیاب دستاویزکے مطابق یہ اہم فیصلے ایف بی آرکے ممبر کسٹمز زاہدکھوکھرکی زیرصدارت اجلاس میں کیے گئے جس میں یہ بھی فیصلہ کیا گیا کہ کسٹمزکراچی کے وی بوک اوراس کے نیٹ ورک ڈیٹا سینٹر کی فعالیت کو برقرار رکھنے کے لیے پاکستان ریونیو آٹومیشن لمیٹڈ کی جانب سے ضرورت کے مطابق بروقت پروکیورمنٹس کو بھی یقینی بنایا جائے گا تاکہ آئندہ کسٹمز کراچی کے وی بوک نیٹ ورک ڈیٹا سینٹر میں بریک ڈاون کی صورتحال سے بچا جا سکے۔  اجلاس میں یہ بھی فیصلہ کیا گیا کہ اربکروڑ روپے سے زائد مالیت کے گڈز ڈکلیئریشن فنڈ کے استعمال کے لیے خصوصی سیکریٹریٹ قائم کیا جائے گا اور یہ سیکریٹریٹ دسمبر کو جاری کردہ کسٹمز جنرل آرڈر نمبر کے مطابق جی ڈی فنڈ کو استعمال میں لائے گا۔ اس بارے میں ایف بی آر حکام کا کہنا ہے کہ ایف بی آر نے  میں درآمدی کنسائنمنٹس کی پروسیسنگ پر  روپے فی کنسائنمنٹ کے حساب سے گڈز ڈ

In [ ]:
import nltk
stopwords=pd.read_csv('/content/drive/MyDrive/stopwords.txt',names=['List'])
# stopwords['List']

stopwords_ur=[]
for li in stopwords['List']:
  stopwords_ur.append(li)
print(len(stopwords_ur))
print(stopwords_ur)

401
['کی', 'ہیں', 'ہے', 'رہا', 'رہی', 'رہے', 'تھا', 'تھے', 'تھی', 'تھى', 'میں', 'کہ', 'کے', 'ہوتے', 'کہہ', 'بنانا', 'پھر', 'لکین', 'ہوتی', 'لیتی', 'ایسی', 'ائے', 'ہوئے', 'ہوۓ ', 'مگر', 'چاہے', 'کیے', 'تاکہ', 'تم', 'آکر', 'لگا', 'ہوگیئں', 'ليے', 'رہتی', 'ہوگئی', 'انھوں', 'چاہتے', 'پاگیئں', 'آنا', 'کروا', 'رکھ', 'آتی', 'یہاں', 'جیسا', 'چکے', 'کرئے', 'دیے', 'چکا', 'ملتا', 'کبھی', 'ایسا', 'کرسکیں', 'ہوسکے', 'سکیں', 'لہذا', 'چاہئے', 'وہیں', 'اٹھایا', 'جہنوں', 'ہمارے', 'لیے', 'آرہے', 'کرگیئں', 'بنانے', 'سکتا', 'وغیرہ', 'دے', 'ہوۓ', 'رہنے', 'ہوۓ', 'کئے', 'لگے', 'لگایا', 'لائے', 'کہے', 'کرے', 'رہئں', 'آگئے', 'کئی', 'کم', 'ملی', 'جنہیں', 'ہوئیں', 'تھیں', 'ابھی', 'پاگئیں', 'آئے', 'کرا', 'دیا', 'جہاں', 'آگئیں', 'کرتی', 'رہیں', 'کرتیں', 'دیتی', 'ہوگئے', 'ديتے', 'انہں', 'ایسے', 'رکھتے', 'رہتے', 'رکھی', 'کیں', 'كیا', 'وه', 'جنہيں', 'کروائی', 'دینا', 'جسے', 'جاتی', 'اسکی', 'ملے', 'کرگئى', 'جن', 'آپکو', 'جنہوں', 'دیئے', 'والی', 'جبکہ', 'دیگر', 'آپکا', 'رکھنے', 'انہىں', 'کيے', 'یعنی', 'كيے', 'بننے', 'ج

# BERTopic Training
The default  bertopic embedding model is paraphrase-multilingual-MiniLM-L12-v2 when selecting language="multilingual". we also performed experiment on 2 another pretained multilingual model from [sentence-tranformer](https://www.sbert.net/docs/pretrained_models.html) and create custom document embedding and passed it to the bertopic model for training.

In [ ]:
#create custom embedding
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
embeddings = model.encode(documents, show_progress_bar=True)
print(embeddings)

.gitattributes:   0%|          | 0.00/690 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.58M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.69k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/341 [00:00<?, ?B/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

[[ 0.05808477  0.0238898   0.00667638 ...  0.00750619  0.05943276
   0.00124432]
 [ 0.00707471  0.06183453  0.01358282 ...  0.03410887  0.04613194
  -0.01144732]
 [-0.04789835  0.05102404 -0.02981449 ...  0.0342866  -0.02764623
   0.00305767]
 ...
 [-0.01387034  0.02076214 -0.04047821 ...  0.00534513 -0.02091605
   0.01629284]
 [-0.05897831  0.0367678   0.03810614 ...  0.06076489 -0.00818423
   0.0283733 ]
 [ 0.0256409  -0.00437732 -0.04543339 ...  0.04750746 -0.03870325
   0.04952908]]


In [ ]:
#pass vectorizer_model to bertopic for stopwords removal
from sklearn.feature_extraction.text import CountVectorizer

vectorizer_model = CountVectorizer(stop_words= stopwords_ur)


In [ ]:
#ClassTfidf for top words
from bertopic.vectorizers import ClassTfidfTransformer

ctfidf_model = ClassTfidfTransformer(bm25_weighting=True, reduce_frequent_words=True)

In [ ]:
#UMAP for dimention reduction
from umap import UMAP
dim_model = UMAP(n_neighbors=10, n_components=5, random_state=42)

In [ ]:
# # #KMeans used for clustering
from sklearn.cluster import KMeans

cluster_model = KMeans(n_clusters=5, random_state=42)

In [ ]:
np.random.seed(42)

In [ ]:
topic_model = BERTopic(language="urdu", low_memory=True ,calculate_probabilities=True, vectorizer_model=vectorizer_model, top_n_words=10, umap_model=dim_model,ctfidf_model=ctfidf_model, hdbscan_model=cluster_model)

In [ ]:
#Fit documents in bertopic
topics, probs = topic_model.fit_transform(documents,embeddings)

In [ ]:
print(probs)

None


In [ ]:
#topics that assign to each document
print(topics)

[2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 3, 4, 4, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 4, 3, 3, 1, 3, 3, 3, 3, 3, 4, 4, 4, 3, 3, 3, 4, 4, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 0, 4, 0, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 1, 3, 3, 1, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 4, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 0, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 

In [ ]:
topic_model.get_topic_freq()

,Topic,Count
2,0,315
1,1,240
0,2,238
3,3,205
4,4,10


In [ ]:
document_topics = topic_model.get_topics()

In [ ]:
#topics with score
print(document_topics)

{0: [('کینسر', 0.3352272309429145), ('استعمال', 0.32220782350239846), ('جسم', 0.2816719065962793), ('علاج', 0.28119033705398294), ('افراد', 0.2793780270480191), ('پانی', 0.2775593680576492), ('ماہرین', 0.26998267574608364), ('صحت', 0.2699432520627466), ('خواتین', 0.2692653392009466), ('تحقیق', 0.2689901864203346)], 1: [('ٹیم', 0.4290244815232503), ('کرکٹ', 0.38164387107811004), ('محمد', 0.35212168612767936), ('میڈل', 0.34896702517259354), ('انگلینڈ', 0.346701053074936), ('گولڈ', 0.345522760815335), ('ٹیسٹ', 0.33846742437071975), ('میچ', 0.33231115158632957), ('کھلاڑیوں', 0.3298867849527424), ('گیمز', 0.32491216264548134)], 2: [('روپے', 0.42167092429937303), ('ارب', 0.3906057822907547), ('فیصد', 0.37385343934555737), ('ٹیکس', 0.3722376228261147), ('حکومت', 0.3670954821509443), ('پاکستان', 0.3630588751443995), ('بجٹ', 0.36087948067480624), ('ڈالر', 0.3563317832608488), ('کروڑ', 0.33605844328636286), ('مالی', 0.3305308801634442)], 3: [('فلم', 0.5785061628508218), ('خان', 0.462910406291833

In [ ]:
#shows which documents assign to specific topic
topic_model.get_representative_docs(topic=None)

{0: [' زندگی کا بنیادی اور اہم جزو پانی ہے زندگی کا آغاز ہی پانی سے ہوتا ہے اسی لیے سائنس داں کسی سیارے کی دریافت پر سب سے پہلے یہ معلوم کرتے ہیں کہ وہاں پانی کے آثار ہیں یا نہیں۔ پانی نہیں تو زندگی بھی نہیں۔  ایک اندازے کے مطابق بقائے انسانی کے لیے ناگزیر اس عظیم نعمت سے دنیا کے  کروڑ  لاکھ افراد محروم ہیں۔ یعنی ہر نو میں سے ایک فرد کو پینے کے لیے صاف پانی میسر نہیں۔ دنیا بھر میں لوگوں تک پینے کے صاف پانی اور سینیٹیشن تک رسائی کے لیے کام کرنے والی غیرسرکاری امریکی تنظیم واٹر ڈاٹ اوآر جی کے اعدادوشمار کے مطابق پانی کی قلت عدم فراہمی اور آلودگی سے سب سے زیادہ عورتوں کے معمولات زندگی متاثر ہوتے ہیں دنیا بھر میں عورتیں روزانہ چھے گھنٹے مختلف علاقوں اور مقامات سے پانی کے حصول پر صرف کرتی ہیں۔ مجموعی طور پر یہ وقت  کروڑ  لاکھ گھنٹے فی دن بنتا ہے۔ ماہرین کے مطابق اگر عورتیں یہ وقت کوئی ہنر سیکھنے یا تعلیم حاصل کرنے میں صَرف کریں تو دنیا کی معاشی ترقی میں اہم تر کردار ادا کرسکتی ہیں۔  عورتوں کے علاوہ گھروں میں بچوں پر بھی پانی بھرنے کی ذمے داری عائد کی جاتی ہے اور اس کے نتیجے میں بچوں کی 

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,315,0_کینسر_استعمال_جسم_علاج,"[کینسر, استعمال, جسم, علاج, افراد, پانی, ماہری...",[ زندگی کا بنیادی اور اہم جزو پانی ہے زندگی کا...
1,1,240,1_ٹیم_کرکٹ_محمد_میڈل,"[ٹیم, کرکٹ, محمد, میڈل, انگلینڈ, گولڈ, ٹیسٹ, م...",[ پھر دوست نے کہا کہ فلاں صحافی کہہ رہے تھے ا...
2,2,238,2_روپے_ارب_فیصد_ٹیکس,"[روپے, ارب, فیصد, ٹیکس, حکومت, پاکستان, بجٹ, ڈ...",[ کراچی وفاقی سیکریٹری تجارت یونس ڈھاگا نے کہ...
3,3,205,3_فلم_خان_ووڈ_بالی,"[فلم, خان, ووڈ, بالی, اداکارہ, سلمان, بھارتی, ...",[ استاد نصرت فتح علیخاں مرحوم کا نام کسی تعار...
4,4,10,4_ظفر_میشا_شفیع_علی,"[ظفر, میشا, شفیع, علی, جنسی, ہراسانی, الزام, گ...",[ چند روز قبل گلوکارہ و اداکارہ میشاشفیع نے ع...


In [ ]:
topic_model.get_document_info(documents)

,Document,Topic,Name,Representation,Representative_Docs,Top_n_words,Representative_document
0,اسلام آباد ایسوسی ایشن آف چارٹرڈ سرٹیفائیڈا...,2,2_روپے_ارب_فیصد_ٹیکس,"[روپے, ارب, فیصد, ٹیکس, حکومت, پاکستان, بجٹ, ڈ...",[ کراچی وفاقی سیکریٹری تجارت یونس ڈھاگا نے کہ...,روپے - ارب - فیصد - ٹیکس - حکومت - پاکستان - ب...,False
1,اسلام آباد ایف بی آرنے اربکروڑ روپے سے زائد...,2,2_روپے_ارب_فیصد_ٹیکس,"[روپے, ارب, فیصد, ٹیکس, حکومت, پاکستان, بجٹ, ڈ...",[ کراچی وفاقی سیکریٹری تجارت یونس ڈھاگا نے کہ...,روپے - ارب - فیصد - ٹیکس - حکومت - پاکستان - ب...,False
2,کراچی بزنس مین گروپ کے چیئرمین اور کراچی چیمب...,2,2_روپے_ارب_فیصد_ٹیکس,"[روپے, ارب, فیصد, ٹیکس, حکومت, پاکستان, بجٹ, ڈ...",[ کراچی وفاقی سیکریٹری تجارت یونس ڈھاگا نے کہ...,روپے - ارب - فیصد - ٹیکس - حکومت - پاکستان - ب...,False
3,کراچی ٹرانسپورٹ اور پٹرولیم مصنوعات کی درآمد...,2,2_روپے_ارب_فیصد_ٹیکس,"[روپے, ارب, فیصد, ٹیکس, حکومت, پاکستان, بجٹ, ڈ...",[ کراچی وفاقی سیکریٹری تجارت یونس ڈھاگا نے کہ...,روپے - ارب - فیصد - ٹیکس - حکومت - پاکستان - ب...,False
4,بجٹ کی آمد آمد ہے۔ ایکسپریس میڈیا گروپ کی ی...,1,1_ٹیم_کرکٹ_محمد_میڈل,"[ٹیم, کرکٹ, محمد, میڈل, انگلینڈ, گولڈ, ٹیسٹ, م...",[ پھر دوست نے کہا کہ فلاں صحافی کہہ رہے تھے ا...,ٹیم - کرکٹ - محمد - میڈل - انگلینڈ - گولڈ - ٹی...,False
...,...,...,...,...,...,...,...
1003,پندرہ سال قبل اس خاتون کا بیٹا ایک حادثے کا ...,0,0_کینسر_استعمال_جسم_علاج,"[کینسر, استعمال, جسم, علاج, افراد, پانی, ماہری...",[ زندگی کا بنیادی اور اہم جزو پانی ہے زندگی کا...,کینسر - استعمال - جسم - علاج - افراد - پانی - ...,False
1004,غیرملکی خبررساں ادارے کے مطابق بھارتی ریاست ...,3,3_فلم_خان_ووڈ_بالی,"[فلم, خان, ووڈ, بالی, اداکارہ, سلمان, بھارتی, ...",[ استاد نصرت فتح علیخاں مرحوم کا نام کسی تعار...,فلم - خان - ووڈ - بالی - اداکارہ - سلمان - بھا...,False
1005,غیرملکی خبررساں ادارے کے مطابق وائٹ ہاؤس کی ...,1,1_ٹیم_کرکٹ_محمد_میڈل,"[ٹیم, کرکٹ, محمد, میڈل, انگلینڈ, گولڈ, ٹیسٹ, م...",[ پھر دوست نے کہا کہ فلاں صحافی کہہ رہے تھے ا...,ٹیم - کرکٹ - محمد - میڈل - انگلینڈ - گولڈ - ٹی...,False
1006,تفصیلات کے مطابق آئرلینڈ کے سالہ مائیکل شر...,0,0_کینسر_استعمال_جسم_علاج,"[کینسر, استعمال, جسم, علاج, افراد, پانی, ماہری...",[ زندگی کا بنیادی اور اہم جزو پانی ہے زندگی کا...,کینسر - استعمال - جسم - علاج - افراد - پانی - ...,False


In [ ]:
topic_distr, _ = topic_model.approximate_distribution(documents, window=3, min_similarity=0.01)

In [ ]:
print(topic_distr)

[[0.21413286 0.18620435 0.455515   0.1027674  0.04138039]
 [0.17070162 0.21392243 0.40156997 0.14043676 0.07336922]
 [0.1459862  0.2435752  0.34943176 0.17615304 0.08485381]
 ...
 [0.1954711  0.3098705  0.22054811 0.17636631 0.09774397]
 [0.35878794 0.17754934 0.16189565 0.18688414 0.11488294]
 [0.18477407 0.25629607 0.11171864 0.2416797  0.20553152]]


In [ ]:
topic_model.visualize_distribution(topic_distr[0], width=600,height=600,  title="Topic Probability Distribution (UDC)")

# Evaluation
we used three evaluation metrics to compare the results.

1. The coherence score is used to capture the degree of similarity between the words within each topic, with higher scores indicating more coherent topics. We used two coherence metrics NPMI and Cv Score.
2. IRBO measures are used to assess how different and distinct the topics are in a topic model.


### Coherence Score
To evaluate the model topics coherence we use [Gensim](https://radimrehurek.com/gensim/models/coherencemodel.html) library

In [ ]:
texts = [[word for word in str(document).split() if word not in stopwords_ur] for document in documents]
id2word = corpora.Dictionary(texts)
corpus = [id2word.doc2bow(text) for text in texts]

In [ ]:
topics_bert=[]
for i in topic_model.get_topics():
  row=[]
  topic= topic_model.get_topic(i)
  for word in topic:
     row.append(word[0])
  topics_bert.append(row)

In [ ]:
print(topics_bert)

[['کینسر', 'استعمال', 'جسم', 'علاج', 'افراد', 'پانی', 'ماہرین', 'صحت', 'خواتین', 'تحقیق'], ['ٹیم', 'کرکٹ', 'محمد', 'میڈل', 'انگلینڈ', 'گولڈ', 'ٹیسٹ', 'میچ', 'کھلاڑیوں', 'گیمز'], ['روپے', 'ارب', 'فیصد', 'ٹیکس', 'حکومت', 'پاکستان', 'بجٹ', 'ڈالر', 'کروڑ', 'مالی'], ['فلم', 'خان', 'ووڈ', 'بالی', 'اداکارہ', 'سلمان', 'بھارتی', 'اداکار', 'میڈیا', 'انڈسٹری'], ['ظفر', 'میشا', 'شفیع', 'علی', 'جنسی', 'ہراسانی', 'الزام', 'گلوکارہ', 'مومنہ', 'ہراساں']]


In [ ]:
# compute Coherence Score

cm = CoherenceModel(topics=topics_bert, texts=texts, dictionary=id2word, coherence='c_v')
coherence = round(cm.get_coherence(),2)
print('\nCoherence Score: ', coherence)


Coherence Score:  0.8


**Diversity Score**

In [ ]:
import itertools
from rbo import rbo
import numpy as np

class InvertedRBO:
    def __init__(self):
        pass

    def irbo(self, topics, topk=10, weight=0.9):
        """
        Calculate inverted Rank Biased Overlap (RBO) as a measure of topic diversity from a list of lists of words.

        :param topics: A list of lists of words representing different topics.
        :param topk: The number of top words on which RBO will be computed.
        :param weight: Weight of each agreement at depth d: p**(d-1). When set to 1.0, there is no weight,
                       and the RBO returns to average overlap.
        :return: The inverted RBO topic diversity score.
        """
        if topk <= 0:
            raise ValueError("topk must be a positive integer.")

        num_topics = len(topics)
        if num_topics == 0:
            raise ValueError("topics list cannot be empty.")

        if topk > len(topics[0]):
            raise Exception('Words in topics are less than topk')

        collect = []
        for list1, list2 in itertools.combinations(topics, 2):
            rbo_val = rbo(list1[:topk], list2[:topk], p=weight)[2]
            collect.append(rbo_val)

        Irbo_score = 1 - np.mean(collect)
        return Irbo_score

In [ ]:
inverted_rbo_calculator = InvertedRBO()
IRBO= round(inverted_rbo_calculator.irbo(topics_bert, topk=10, weight=0.9),2)
print("Inverted RBO Score:", IRBO)

Inverted RBO Score: 1.0


# Visualize Topics

In [ ]:
topic_model.visualize_topics()

In [ ]:
topic_model.visualize_barchart(n_words=10, width=220, height=270, title= "Topic Word Scores (UDC)")

# Model serialization

In [ ]:
# Save model
topic_model.save("my_model")

2023-12-29 00:04:00,067 - BERTopic - WARNING: When you use `pickle` to save/load a BERTopic model,please make sure that the environments in which you saveand load the model are **exactly** the same. The version of BERTopic,its dependencies, and python need to remain the same.


In [ ]:
loaded_topic_model = topic_model.load("my_model")
loaded_topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,0,315,0_کینسر_استعمال_جسم_علاج,"[کینسر, استعمال, جسم, علاج, افراد, پانی, ماہری...",[ زندگی کا بنیادی اور اہم جزو پانی ہے زندگی کا...
1,1,240,1_ٹیم_کرکٹ_محمد_میڈل,"[ٹیم, کرکٹ, محمد, میڈل, انگلینڈ, گولڈ, ٹیسٹ, م...",[ پھر دوست نے کہا کہ فلاں صحافی کہہ رہے تھے ا...
2,2,238,2_روپے_ارب_فیصد_ٹیکس,"[روپے, ارب, فیصد, ٹیکس, حکومت, پاکستان, بجٹ, ڈ...",[ کراچی وفاقی سیکریٹری تجارت یونس ڈھاگا نے کہ...
3,3,205,3_فلم_خان_ووڈ_بالی,"[فلم, خان, ووڈ, بالی, اداکارہ, سلمان, بھارتی, ...",[ استاد نصرت فتح علیخاں مرحوم کا نام کسی تعار...
4,4,10,4_ظفر_میشا_شفیع_علی,"[ظفر, میشا, شفیع, علی, جنسی, ہراسانی, الزام, گ...",[ چند روز قبل گلوکارہ و اداکارہ میشاشفیع نے ع...
